##  This is a script to use deep learning to predict stuff

# Do Imports

In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score
import math

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import matplotlib.pyplot as plt

# Read data and create Dataset iterators

In [2]:
# Read in data (from pkl file probably)

#os.getcwd()
df = pd.read_pickle('../data/price_level_total_view_2017-01-03_AAPL_grouped_2')
df.head()
df.tail()

,trade_volume,mid_price_log,target,trade_volume_differential,mid_price_log_direction_0_1,mid_price_log_direction_0_2,mid_price_log_direction_0_3,mid_price_log_direction_0_4,mid_price_log_direction_0_5,mid_price_log_direction_0_6,trade_volume_differential_direction_0_1,trade_volume_differential_direction_0_2,trade_volume_differential_direction_0_3,trade_volume_differential_direction_0_4,trade_volume_differential_direction_0_5,trade_volume_differential_direction_0_6
groupid,,,,,,,,,,,,,,,,
57599089,4659293,25.491830,2,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599090,4659293,25.550179,0,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599091,4659293,25.372644,2,0.0,0,0,0,0,0,0,1,1,1,1,1,1
57599092,4659293,25.571578,0,0.0,2,2,2,2,2,2,1,1,1,1,1,1
57599093,4659293,24.424568,0,0.0,0,0,0,0,0,0,1,1,1,1,1,1


In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
dfX = df.iloc[:,[1,3,4,5,6,10,11,12]]

#dfX = ss.fit_transform(dfX)
#dfX = df.iloc[1:,[1,3,4,5,6,10,11,12]]
dfy = df.iloc[:,2]

# Define functions

In [5]:
def features_2d_to_3d(data, labels, window):
    data_n, data_w = data.shape
    stride1, stride2 = data.strides
    new_len = data_n - window
    data3d = as_strided(data, [new_len , window, data_w], strides=[stride1, stride1, stride2])
    return(data3d, labels[:len(labels)-window])

def flatten_3d(data):
    data_n = data.shape[0]
    new_width = data.shape[1]*data.shape[2]
    
    return np.reshape(data, (data_n, new_width))
    
def split_data(dfX, dfy, train_frac):
    
    #X = df.iloc[:,0:2]
    #y = df.iloc[:,2:3]
    X = dfX
    y = dfy
    n = X.shape[0]
    cutoff = np.floor(n * train_frac).astype(int) # total - the number you want to test, which here i'm flooring 
    #                   (amount you want in training should be 1/10th value the denominator)
    # cutoff
    
    #print(cutoff)

    X_train, X_test = (X.iloc[0:cutoff , :] , X.iloc[cutoff: , :] )
    #print(X_train.shape, X_test.shape)

    y_train, y_test = (y.iloc[0:cutoff].values.ravel() , y.iloc[cutoff:].values.ravel() )
    
    ss = StandardScaler()
    ss.fit(X_train)
    X_train = ss.transform(X_train)
    X_test = ss.transform(X_test)
    
    
#     y_train_temp = np.zeros([y_train.shape[0], 3])
#     y_train_temp[np.arange(y_train.shape[0]),y_train] = 1
#     y_train = y_train_temp.astype(int)
    
#     y_test_temp = np.zeros([y_test.shape[0], 3])
#     y_test_temp[np.arange(y_test.shape[0]),y_test] = 1
#     y_test = y_test_temp.astype(int)

    
    return X_train, y_train, X_test, y_test


# Define model

# Train model

In [6]:
# Create train/test sets
trainarray,labelarray,testarray,testlabelarray = split_data(dfX, dfy, 0.8)

In [7]:
# transform data set to use data windows
from numpy.lib.stride_tricks import as_strided
window_size = 6

trainarray, labelarray = features_2d_to_3d(np.array(trainarray), np.array(labelarray), window_size)
testarray, testlabelarray = features_2d_to_3d(np.array(testarray), np.array(testlabelarray), window_size)

labelarray = labelarray[window_size:]
testlabelarray = testlabelarray[window_size:]
trainarray = trainarray[:(trainarray.shape[0]-window_size),]
testarray = testarray[:(testarray.shape[0]-window_size),]

# Input
height = trainarray.shape[1]
width = trainarray.shape[2]
channels = 1
n_inputs = height * width

trainarray = flatten_3d(trainarray) # don't need to flatten for CNN
testarray = flatten_3d(testarray)

In [8]:
import os
import numpy as np
import tensorflow as tf

# Step 1: Define parameters for the CNN



# Parameters for TWO convolutional layers:
# Number of feature maps, size of receptive field, stride, and padding
conv1_fmaps, conv1_ksize, conv1_stride, conv1_pad = 32, 3, 1, "SAME"
conv2_fmaps, conv2_ksize, conv2_stride, conv2_pad = 64, 3, 1, "SAME"

# Define a pooling layer
pool3_dropout_rate = 0.25
pool3_fmaps = conv2_fmaps

# Define a fully connected layer
n_fc1 = 128
fc1_dropout_rate = 0.5

# Output
n_outputs = 3

tf.reset_default_graph()

# Step 2: Set up placeholders for input data
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

# Step 3: Set up the two convolutional layers using tf.layers.conv2d
# Hint: arguments would include the parameters defined above, and what activation function to use
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                         strides=(conv1_stride), padding=conv1_pad, activation=tf.nn.relu,
                         name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize, 
                         strides=(conv2_stride), padding=conv2_pad, activation=tf.nn.relu,
                         name="conv2")

# Step 4: Set up the pooling layer with dropout using tf.nn.max_pool

#4-d tensor is [batch, height, width, channel] -- no one does pooling across the batch apparently, so keep that 1 always
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID", name="pool")    
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * height//2 * width//2])
    pool3_flat_drop = tf.layers.dropout(pool3_flat, pool3_dropout_rate, training=training)

# Step 5: Set up the fully connected layer using tf.layers.dense
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

# Step 6: Calculate final output from the output of the fully connected layer
with tf.name_scope("output"):
    logits, Y_proba = None, None # Hint: "Y_proba" is the softmax output of the "logits"
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

# Step 5: Define the optimizer; taking as input (learning_rate) and (loss)
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

# Step 6: Define the evaluation metric
with tf.name_scope("eval"):
    correct, accuracy = None, None
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_sum(tf.cast(correct, tf.float32))
    
# Step 7: Initiate
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

# Step 8: Read in data
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
# X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
# X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
# y_train = y_train.astype(np.int32)
# y_test = y_test.astype(np.int32)
# X_valid, X_train = X_train[:5000], X_train[5000:]
# y_valid, y_train = y_train[:5000], y_train[5000:]

X_train = trainarray.astype(np.float32).reshape(-1, height*width)
y_train = labelarray.astype(np.int32)
X_valid = testarray.astype(np.float32).reshape(-1, height*width)
y_valid = testlabelarray.astype(np.int32)
X_t = X_valid

# Step 9: Define some necessary functions
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

# Step 10: Define training and evaluation parameters
n_epochs = 100
batch_size = 50
iteration = 0

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None

# Step 11: Train and evaluate CNN with Early Stopping procedure defined at the very top
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            iteration += 1
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: X_valid, y: y_valid})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})/batch_size
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})/44000
        print("Epoch {}, last batch accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_batch * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)

Epoch 0, last batch accuracy: 60.0000%, valid. accuracy: 74.9705%, valid. best loss: 0.996817
Epoch 1, last batch accuracy: 46.0000%, valid. accuracy: 74.9273%, valid. best loss: 0.996236
Epoch 2, last batch accuracy: 46.0000%, valid. accuracy: 74.9682%, valid. best loss: 0.996236
Early stopping!


NameError: name 'X_test' is not defined

# Do Evaluation

In [ ]:
# Make predictions
p = get_predictions(tep_init, test_n, output)

# actual along left, predicted along top
confusion_matrix(testlabelarray, p)


In [ ]:
print(p[:10])
print(testlabelarray[:10])

In [ ]:
accuracy_score(testlabelarray, p)